In [2]:
# %pip install pandas
# %pip install numpy
# %pip install opencv-python
# %pip install sklearn
# %pip install tensorflow
# %pip install joblib
# %pip install os
# %pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) 
import matplotlib
import matplotlib.pyplot as plt
import os
import cv2
import PIL
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import joblib

2022-10-31 23:20:57.391819: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-31 23:20:57.515466: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gerry/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-10-31 23:20:57.515484: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-31 23:20:57.542865: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already b

In [9]:
#inputs to the script

#Model Path
model_dir_path="./model/"


all_regions_dir_path="../static/All_Regions/"


label_encoder_path="./label_encoder.joblib"

In [10]:
model= keras.models.load_model(model_dir_path)
label_encoder = joblib.load(label_encoder_path)

OpError: ./model/variables/variables.data-00000-of-00001; No such file or directory

In [ ]:
def animal_identifier(region_dir,model):
    df=pd.DataFrame(columns=['Image_Name','Region_Name','Species'])
    for i in range(0,len(os.listdir(region_dir))):
        images=[]
        img = cv2.imread(region_dir+'/'+os.listdir(region_dir)[i])
        resized_img = cv2.resize(img,(224,224))
        resized_img = resized_img / 255.0
        images.append(resized_img)
        images = np.array(images,dtype = 'float32')
        preds = model.predict(images)
        preds = np.argmax(preds,axis = 1)
        pred_animal=label_encoder_path.inverse_transform([preds[0]])[0]
        df.loc[i,'Image_Name']=os.listdir(region_dir)[i]
        df.loc[i,'Region_Name']=region_dir.split('\\')[-1]
        df.loc[i,'Species']=pred_animal
    return df

In [ ]:
finaldf=pd.DataFrame(columns=['Image_Name','Region_Name','Species'])

def classify_animals_for_all_regions():
    for i in range(0,len(os.listdir(all_regions_dir_path))):
        region_dir=all_regions_dir_path+'\\'+os.listdir(all_regions_dir_path)[i]
        preds=animal_identifier(region_dir,model)
        finaldf=finaldf.append(preds,ignore_index=True)
        finaldf.to_csv('Region_Wise_Predictions.csv',index=False)

In [ ]:
# Run classifier
classify_animals_for_all_regions()